# Vector indexing on Google Colab

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install langchain

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install ipython-autotime
%load_ext autotime

time: 246 µs (started: 2024-01-10 15:01:28 +00:00)


In [ ]:
!pip install faiss-gpu

time: 4.72 s (started: 2024-01-10 15:01:28 +00:00)


In [ ]:
import os
import json
from pathlib import Path
from pprint import pprint

from bs4 import BeautifulSoup

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

time: 758 ms (started: 2024-01-10 15:01:32 +00:00)


In [ ]:
# file_path='./drive/MyDrive/vectordb/output_grca_clean.json'
file_path='./drive/MyDrive/vectordb/output_all_parks_clean.json'

data = json.loads(Path(file_path).read_text())
# Inspect visually the loaded data
data[0]
# Use Beautiful Soup to format html into txt
data_txt = []
for d in data:
    data_txt.append(
        "TITLE: "+d['title']+" \n"+
        "URL: "+d['url']+" \n"+
        "HTML: "+ BeautifulSoup(d['html']).get_text()
    )
# Inspect the formatted data
pprint(data_txt[0])

<ipython-input-7-2b5870f31ec8>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  "HTML: "+ BeautifulSoup(d['html']).get_text()


('TITLE: Teacher Resources: Chapter 13 - Island of the Blue Dolphins (U.S. '
 'National Park Service) \n'
 'URL: https://www.nps.gov/subjects/islandofthebluedolphins/teacher-13.htm \n'
 'HTML: Opinion: Think about ways men and women on San Nicolas Island might '
 'have shared the division of labor. Do you agree or disagree with the custom '
 'that women in Karana’s tribe were not allowed to make weapons? Remember to '
 'consider what it might have been like to live in her particular society. Use '
 'reasons and information to support your point of view.\n'
 'Informative/explanatory: Describe the mating and breeding habits of northern '
 'elephant seals. Gather information from Voices from the Field to develop the '
 'topic. Draw evidence from informational text.\n'
 'Narrative: Imagine you are Karana watching a fight between two bull elephant '
 'seals. Write a narrative describing the battle. Include details about what '
 'you might see, hear, and smell.\n'
 'An official form of the U

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
# Convert plain text to documents
docs = text_splitter.create_documents(data_txt)
# Inspect the documents
docs[3:10]
pprint(docs[2].to_json()['kwargs']['page_content'])

('TITLE: Teacher Resources: Chapter 11 - Island of the Blue Dolphins (U.S. '
 'National Park Service) \n'
 'URL: https://www.nps.gov/subjects/islandofthebluedolphins/teacher-11.htm \n'
 'HTML: Opinion: What factors did Karana consider in choosing the location for '
 'her new home? Would you have chosen the same location? Why or why not? Use '
 'evidence from the story to support your claim.\n'
 'Informative/explanatory: Describe some of the ways Karana protects her food '
 'and possessions from wild animals. Develop the topic with facts and other '
 'information related to the topic.\n'
 'Narrative: Karana plans to kill the wild dogs. Imagine you are a wild dog. '
 'How might you react when Karana approaches your cave? Use sensory details to '
 'convey experiences and events precisely.\n'
 'An official form of the United States government. Provided by Touchpoints\n'
 'Download the official NPS app before your next visit')
time: 17.8 s (started: 2024-01-10 15:02:00 +00:00)


time: 17.8 s (started: 2024-01-10 15:02:00 +00:00)


time: 17.8 s (started: 2024-01-10 15:02:00 +00:00)


In [ ]:
# !ls drive/MyDrive/vectordb/output_grca_clean.json
!ls drive/MyDrive/vectordb

output_all_parks_clean.json  output_grca_clean.json
time: 106 ms (started: 2024-01-10 15:02:18 +00:00)


In [ ]:
## Build the vector database
modelPath = "all-MiniLM-L6-v2"
model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings':False}

embeddings = HuggingFaceEmbeddings(
  model_name = modelPath,
  model_kwargs = model_kwargs,
  encode_kwargs=encode_kwargs
)

time: 3.02 s (started: 2024-01-10 15:02:18 +00:00)


In [ ]:
# db = FAISS.from_documents(docs, embeddings)
db = await FAISS.afrom_documents(docs, embeddings)

time: 8min 27s (started: 2024-01-10 15:02:21 +00:00)


In [ ]:
db.save_local("faiss_index")

time: 4.58 s (started: 2024-01-10 15:10:49 +00:00)
